In [1]:
import torch
import pandas as pd
from torch.utils.data import Dataset, random_split, DataLoader
from torch import nn
from torch import optim

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
df = pd.read_parquet("train/data/data.parquet")

In [4]:
def get_value(entry, category):
    value = entry[0].get(category)

In [5]:
class ParquetDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, parquet_file, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.signal = pd.read_parquet(parquet_file)['FatJets']
        self.transform = transform

    def __len__(self):
        return len(self.signal)

    def __getitem__(self, idx):
        return [self.signal[idx][0]['msoftdrop']]

In [6]:
dataset = ParquetDataset("train/data/data.parquet")

In [7]:
train, test = random_split(dataset, [int(len(dataset)*0.8), int(len(dataset)*0.2)])

In [8]:
train_dl = DataLoader(train, batch_size=32, shuffle=True)

In [9]:
test_dl = DataLoader(test, batch_size=1024, shuffle=False)

In [10]:
# model definition
class MLP(nn.Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        self.layer = nn.Linear(n_inputs, 1)
        self.activation = nn.Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        X = self.layer(X)
        X = self.activation(X)
        return X

In [11]:
model = MLP(34).to(device)
print(model)

MLP(
  (layer): Linear(in_features=34, out_features=1, bias=True)
  (activation): Sigmoid()
)


In [12]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [13]:
for i, j in enumerate(train_dl):
    print(j)

[tensor([  5.0312,  59.5000,  68.3750,  -1.0000, 108.2500,  25.1250,  82.5625,
        100.5000,  97.6250, 103.6250,  -1.0000,  70.1250, 114.3750, 135.1250,
         43.3438,  59.2812,   6.4727,   3.8945,  27.7500,  23.4531,  21.3750,
        113.6250,  94.5625,  50.0000,  63.2500,  13.8672,  35.1875,  76.5000,
         37.9375,  17.9375,  84.0000, 102.9375], dtype=torch.float64)]
[tensor([ 27.9531, 124.6875,  60.5938,  60.5000,   3.5410, 102.7500, 111.9375,
         50.2812,  62.2500, 128.0000,  76.0000, 126.9375,  16.6875,   4.6484,
          1.9775,  51.2812,  46.7188, 141.3750,  56.7188,   5.0469,  66.8125,
          5.0742,   2.9961,  55.1250,  17.2969,  61.7812, 123.3125,  62.8125,
         47.9688, 119.4375, 135.8750,  51.1250], dtype=torch.float64)]
[tensor([  2.1895,  29.7031,  14.1094,   6.2695,  80.2500, 104.6875,   2.2207,
         80.5625, 144.5000,  11.9688,  82.5625, 145.5000,  -1.0000,  83.3750,
         23.0625,   1.2207,  93.3750,  23.7188,   1.8008,  27.0000,   3.119

In [14]:
for i, (inputs, targets) in enumerate(train_dl):
    print(i)
    # clear the gradients
#    optimizer.zero_grad()
#     # compute the model output
#     yhat = model(inputs)
#     # calculate loss
#     loss = criterion(yhat, targets)
#     # credit assignment
#     loss.backward()
#     # update model weights
#     optimizer.step()

ValueError: not enough values to unpack (expected 2, got 1)

In [45]:
def get_value(entry, category):
    value = entry[0].get(category)
    return value
df['FatJets'].apply(lambda x: get_value(x, 'msoftdrop'))

0        82.062500
1        96.375000
2        30.390625
3       101.812500
4       111.625000
           ...    
1935     86.000000
1936    146.000000
1937      3.289062
1938     56.625000
1939     35.187500
Name: FatJets, Length: 1940, dtype: float64